In [2]:
with open("the-verdict (1).txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("total no of chars:", len(raw_text))
print(raw_text[:99])

total no of chars: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


We use re to split the words and expressions in the text

In [9]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Creating Token IDs

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer, token in enumerate(all_words)} ##U are creating a dic, and assigning numbers from 0 to so on to alpabetically arranged words


In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=30:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)


## Simple tokenizer class

In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?!"()\')]|--|\s)', text)
        
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text= " ".join([self.int_to_str[i] for i in ids])
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        return text

In [16]:
tokenizer = SimpleTokenizerV1(vocab)

text = """I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory"""
#use 3 """
ids = tokenizer.encode(text)
print(ids)

[53, 44, 149, 1003, 57, 38, 818, 115, 256, 486, 6, 1002, 115, 500, 435, 392, 6, 908, 585, 1077, 709, 508, 961, 1016, 663, 1016, 535, 987, 5, 568, 988, 538, 722, 549, 496]


In [17]:
tokenizer.decode(ids)

'I HAD always thought Jack Gisburn rather a cheap genius -- though a good fellow enough -- so it was no great surprise to me to hear that, in the height of his glory'

### Adding special text tokens

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [26]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [27]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?!"()\')]|--|\s)', text)
        
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text= " ".join([self.int_to_str[i] for i in ids])
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        return text
    

In [32]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [33]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [35]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## Byte Pair Encoding

In [38]:
! pip3 install tiktoken

In [39]:
import importlib
import tiktoken


In [42]:
tokenizer = tiktoken.get_encoding("gpt2") # this is BPE from gpt2

In [43]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
    )

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [44]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


## CREATING INPUT - TARGET PAIRS

In [45]:
with open("the-verdict (1).txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
enc_text = tokenizer.encode(raw_text) # we tokenize entire verdict text using the BPE tokenizer
print(len(enc_text))

5145


In [46]:
#Remove last 50 tokens to make things interesting
enc_sample = enc_text[50:]

In [50]:
context_size = 4

x = enc_sample[:context_size] #takes first 4 elements
y = enc_sample[1: context_size+1]

print(f"x: {x}")
print(f"y:      {y}" )

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [51]:
#to show how these are as input-target pairs

for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    
    print(context, "--->", desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


In [52]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    
    print(tokenizer.decode(context), "--->", tokenizer.decode([desired]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


## Implementing a Data Loader 

In [54]:
! pip3 install torch

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 1.0/212.5 MB 31.1 MB/s eta 0:00:07
   ---------------------------------------- 2.0/212.5 MB 21.4 MB/s eta 0:00:10
    --------------------------------------- 4.0/212.5 MB 25.3 MB/s eta 0:00:09
   - -------------------------------------- 5.9/212.5 MB 29.2 MB/s eta 0:00:08
   - -------------------------------------- 7.6/212.5 MB 30.2 MB/s eta 0:00:07
   - -------------------------------------- 8.8/212.5 MB 31.1 MB/s eta 0:00:07
   -- ------------------------------------- 11.2/212.5 MB 31.2 MB/s eta 0:00:07
   -- ------------------------------------- 13.1/212.5 MB 38.6 MB/s eta 0:00:06
   -- ------------------------------------- 15.2/212.5 MB 38.6 MB/s eta 0:00:06
   --- ------------------------------------ 17.2/212.5 MB 38.5 MB/s eta 0:00:06
   --- ------------------------------------ 19.6/212.5 MB 43.5 MB/s eta 0:00:05
   --- ------------------------------------ 20.7/212.5 

In [55]:
#we use pytorch for this

from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        #Tokenize entire text
        
        token_ids = tokenizer.encode(txt,allowed_special = {"<|endoftext|>"})
        
        #use a sliding window to chunk the book into overlapping sequences of max_length
        
        for i in range(0,len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
        
        

In [62]:
def create_dataloader_v1(txt, batch_size = 4, max_length=256, stride = 128,
                         shuffle = True, drop_last=True, num_workers=0):
    
    #initalise the tokeniser
    
    tokenizer = tiktoken.get_encoding("gpt2")
    
    #Creating dataset
    
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    #Creating dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers=num_workers
    )
    
    return dataloader


In [63]:
with open("the-verdict (1).txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [64]:
#Convert dataloader into a Python iterator to fetch the next entry via Python's built-in next() function

In [65]:
import torch
print("Pytorch version:", torch.__version__)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter =iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

Pytorch version: 2.7.0+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


## Creating Token Embeddings

In [66]:
input_ids = torch.tensor([2,3,5,1])


In [70]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123) # Sets the random seed so results are reproducible 
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [71]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [72]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [73]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


# POSITIONAL EMBEDDINGS

In [74]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [76]:
#Initaite the dataloader

max_length = 4 #context_size is 4
dataloader = create_dataloader_v1(
    raw_text, batch_size = 8, max_length = max_length,
    stride = max_length, shuffle = False
)
data_iter = iter(dataloader) #iterate through the dataloader
inputs, targets = next(data_iter)

In [78]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [81]:
#so now the token id tensor is 8*4 dim, we need to transform or embedd these token ids into 256 dim vectors
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [82]:
# abs positional embedding
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)


In [84]:
#we only need 4 (max_length ) positional embedding vectors
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


# Input Embeddings = Pos + Token Embeddings

In [85]:
#addition...
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


# IMPLEMENTING A SIMPLIFIED ATTENTION MECHANISM


In [4]:
import torch

inputs = torch.tensor(
    [[0.43,0.15,0.89], #Your
     [0.55,0.87,0.66], #journey
     [0.57,0.85,0.64], #starts
     [0.22,0.58,0.33], #with
     [0.77,0.25,0.10], #one
     [0.05, 0.80, 0.55]]s
)                       #step

In [ ]:
import torch

inputs = torch.tensor(
    [[0.43,0.15,0.89], #Your
     [0.55,0.87,0.66], #journey
     [0.57,0.85,0.64], #starts
     [0.22,0.58,0.33], #with
     [0.77,0.25,0.10], #one
     [0.05, 0.80, 0.55]]
)

#step
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


words = ['Your', 'Journey', 'starts', 'with', 'one', 'step']

x_coords = inputs[:, 0].numpy()
y_coords = inputs[:, 1].numpy()
z_coords = inputs[:, 2].numpy()
#create 3d plot

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

for x,y,z, word in zip(x_coords, y_coords, z_coords, words):
    ax.scatter(x,y,z)
    ax.text(x,y,z, word, fontsize = 10)
    
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.title('3D Plot of Word Embeddings')
plt.show()
